# Algorithm Implementation

In [30]:
from collections import defaultdict
import heapq
import math

# Graph data structure #
class Graph:
  def __init__(self):
    self.adjacency_list = DefaultDict(list) # Node -> [(neighbor, weight)...]
    self.coordinates = {}  # Node -> (x, y)

  # Add a weighted edge
  def add_edge(self, node1, node2, weight):
    self.adjacency_list[node1].append((node2, weight))
    self.adjacency_list[node2].append((node1, weight))

  # Add node coordinates
  def add_node_coordinates(self, node, x, y):
    self.coordinates[node] = (x, y)

  # Get neighbors for a node
  def get_neighbors(self, node):
    neighbors = self.adjacency_list[node]
    return sorted(neighbors, key=lambda x: int(x[0].split('_')[1])) # Sort nodes by their ID

  # Get all nodes in the graph
  def get_nodes(self):
    nodes = list(self.adjacency_list.keys())
    return sorted(nodes, key=lambda x: int(x.split('_')[1])) # Sort nodes by their ID

  # Euclidean distance heuristic
  def euclidean_distance(self, node1, node2):
    if node1 not in self.coordinates or node2 not in self.coordinates:
      raise ValueError("Nodes must have coordinates")

    x1, y1 = self.coordinates[node1]
    x2, y2 = self.coordinates[node2]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

  # Manhattan Distance heuristic
  def manhattan_distance(self, node1, node2):
    if node1 not in self.coordinates or node2 not in self.coordinates:
      raise ValueError("Nodes must have coordinates")

    x1, y1 = self.coordinates[node1]
    x2, y2 = self.coordinates[node2]
    return abs(x2 - x1) + abs(y2 - y1)


# Function to parse input files
def parse_input(edge_file, node_file):

  graph = Graph()

  with open(edge_file) as f:
    for line in f:
      n1, n2, weight = line.strip().split(',')
      graph.add_edge(n1.strip(), n2.strip(), float(weight.strip()))

  with open(node_file) as f:
    for line in f:
      node, x, y = line.strip().split(',')
      node = node.strip()
      x = x.strip()
      y = y.strip()
      graph.add_node_coordinates(node, float(x), float(y))

  return graph

The above code implements a graph data structure with the graph represented as an adjacency list. Each graph object has a dictionary for the adjacency list and an array which contains each node and its associated coordinates. The class has functions for adding edges and node coordinates as well as getting the neighbors of a given node.

\

An adjacency list was chosen as the data structure to represent the graph because it is highly suitable for search tasks, allowing for access of a node's neighbors in O(1) time.

\

The get_neighbors() and get_nodes() functions return their lists of nodes in sorted order, which makes the search algorithms choose the lowest key value when deciding on a frontier node to visit.

\
Here, the heuristic functions for the A* algorithm are also defined, which will be discussed in more detail below.


In [31]:
def dfs(graph, start, end):
  stack = [start]
  path = []

  while stack:
    node = stack.pop()
    path.append(node)

    if node == end:
      return path

    for neighbor, _ in reversed(graph.get_neighbors(node)):
      if neighbor not in path:
        stack.append(neighbor)
  return path

Here, the depth-first search function is defined. The function uses a stack to represent the frontier nodes, and searches them in numerical order with a last in, first out selection procedure. When there are multiple ndoes on the frontier, it chooses the lowest ordered node first. The implementation for this ordering is accomplished by reversing the numerical ordering returned by get_neighbors(), which ensures that lower order nodes are placed on top of the stack.

In [32]:
def bfs(graph, start, end):

  queue = [start]
  visited = set([start])
  path = []

  while queue:
    node = queue.pop(0)
    path.append(node)

    if node == end:
      return path


    for neighbor, _ in (graph.get_neighbors(node)):
      if neighbor not in visited:
        visited.add(neighbor)
        queue.append(neighbor)

  return path

Above is the implementation of the breadth-first search algorithm. It behaves similarly to the depth first search algorithm, with the only difference being that a queue is used to represent the frontier nodes as opposed to a stack. It uses a queue with the first in, first out procedure for selection of the next node at each step.

In [33]:

def a_star(graph, start, end, heuristic):

    def calculate_heuristic(node):
        if heuristic == 'euclidean':
            return graph.euclidean_distance(node, end)
        elif heuristic == 'manhattan':
            return graph.manhattan_distance(node, end)
        else:
            return 0

    def process_neighbor(neighbor, weight, current, g_score, f_score, came_from, open_set):
        tentative_g_score = g_score[current] + weight
        if tentative_g_score < g_score[neighbor]:
            came_from[neighbor] = current
            g_score[neighbor] = tentative_g_score
            f_score[neighbor] = tentative_g_score + calculate_heuristic(neighbor)
            heapq.heappush(open_set, (f_score[neighbor], neighbor))

    open_set = [(0, start)]  # (f_score, node)
    closed_set = set()
    came_from = {}
    visited_order = []

    g_score = defaultdict(lambda: float('inf'))
    g_score[start] = 0

    f_score = defaultdict(lambda: float('inf'))
    f_score[start] = calculate_heuristic(start)

    while open_set:
        _, current = heapq.heappop(open_set)
        if current in closed_set:
            continue

        visited_order.append(current)
        closed_set.add(current)

        if current == end:
            return visited_order

        for neighbor, weight in graph.get_neighbors(current):
            if neighbor in closed_set:
                continue
            process_neighbor(neighbor, weight, current, g_score, f_score, came_from, open_set)

    return visited_order


Above is the implementation of the a-star algorithm. It uses two different heuristics to evaluate distance to the goal: Euclidean distance and Manhattan distance. The algorithm maintains an open set of nodes to be evaluated and a closed set of nodes already processed. During each iteration, the algorithm selects the node with the lowest f-score from the open set using a priority queue implemented using python's heapq module. If it is the goal node, the search ends. If not, the current node is moved from the open set to the closed set to avoid reprocessing, and its neighbors are processed. Helper functions are used to make the code more readable and avoid writing two different A* functions for the different search heuristics.
\
\
**Euclidean Distance Heuristic**
\
This was a clear option for search heuristic because each node has a 2-D coordinate associated with it. This is an admissible heuristic because the straight line distance from any node N to the goal node is always less than or equal to the distance of the real optimal path. This is because the shortest distance between two points is always a straight line. This helps A* perform better than the uninformed search because it gives a direction to traverse into rather than searching all directions equally.
\
\
**Manhattan Distance Heuristic**
\
This heuristic was chosen also because it fits the problem domain quite well, much like Euclidean distance but more bounded. Since each node has a 2-D coordinate, and moves can only be made to adjacent nodes in cardinal directions, we can use the Manhattan distance to estimate the distance from each node to the goal node. This heuristic is admissible because it always takes the minimum number of steps on a grid from one node to another, which is never greater than the real value. This helps A* perform better than the uninformed search algorithms because it provides a grid direction in which to traverse and matches the movement constraints of the problem.


# Evaluation and Analysis

Two evaluation metrics will be used to test the performance of these search algorithms:


1.   Time taken to complete each test case
2.   Number of steps taken, as defined by number of nodes visited

In [38]:
# Time measurements #

if __name__ == '__main__':

  print("Test Case 1: 25 Nodes")
  graph = parse_input("TestCase_01_EdgeList.txt", "TestCase_01_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print("DFS time:")
  time_dfs = %timeit -o dfs(graph, start, end)
  print("\nBFS time:")
  time_bfs = %timeit -o bfs(graph, start, end)
  print("\nA* Euclidean time:")
  time_a_star_euclidean = %timeit -o a_star(graph, start, end, 'euclidean')
  print("\nA* Manhattan time:")
  time_a_star_manhattan = %timeit -o a_star(graph, start, end, 'manhattan')

  print("\nTest Case 2: 100 Nodes")
  graph = parse_input("TestCase_02_EdgeList.txt", "TestCase_02_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print("DFS time:")
  time_dfs = %timeit -o dfs(graph, start, end)
  print("\nBFS time:")
  time_bfs = %timeit -o bfs(graph, start, end)
  print("\nA* Euclidean time:")
  time_a_star_euclidean = %timeit -o a_star(graph, start, end, 'euclidean')
  print("\nA* Manhattan time:")
  time_a_star_manhattan = %timeit -o a_star(graph, start, end, 'manhattan')

  print("\nTest Case 3: 1000 Nodes")
  graph = parse_input("TestCase_03_EdgeList.txt", "TestCase_03_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print("DFS time:")
  time_dfs = %timeit -o dfs(graph, start, end)
  print("\nBFS time:")
  time_bfs = %timeit -o bfs(graph, start, end)
  print("\nA* Euclidean time:")
  time_a_star_euclidean = %timeit -o a_star(graph, start, end, 'euclidean')
  print("\nA* Manhattan time:")
  time_a_star_manhattan = %timeit -o a_star(graph, start, end, 'manhattan')

Test Case 1: 25 Nodes
DFS time:
51.9 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
BFS time:
35 µs ± 8.23 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
A* Euclidean time:
44 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
A* Manhattan time:
42 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Test Case 2: 100 Nodes
DFS time:
179 µs ± 47.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
BFS time:
121 µs ± 32 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
A* Euclidean time:
162 µs ± 39.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
A* Manhattan time:
144 µs ± 35.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Test Case 3: 1000 Nodes
DFS time:
3.73 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
BFS time:
1.86 ms ± 458 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
A* Euclidean time:
2.94 ms ± 90.8 µs per loop (mean ± std. dev. of 7 r

It appears that in all test cases, the depth-first search was the slowest of the three algorithms on average. This makes sense because it is the one that is most likely to take a wrong path. Breadth-first search performed the quickest of the three in all cases, especially those with more nodes. This makes sense because it requires less exploration than DFS, and has no need to calculate heuristics as with A*.

In [42]:
# Number of steps measurements #
if __name__ == '__main__':

  print("Test Case 1: 25 Nodes")
  graph = parse_input("TestCase_01_EdgeList.txt", "TestCase_01_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print(len(dfs(graph, start, end)))
  print(len(bfs(graph, start, end)))
  print(len(a_star(graph, start, end, 'euclidean')))
  print(len(a_star(graph, start, end, 'manhattan')))

  print("\nTest Case 2: 100 Nodes")
  graph = parse_input("TestCase_02_EdgeList.txt", "TestCase_02_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print(len(dfs(graph, start, end)))
  print(len(bfs(graph, start, end)))
  print(len(a_star(graph, start, end, 'euclidean')))
  print(len(a_star(graph, start, end, 'manhattan')))

  print("\nTest Case 3: 1000 Nodes")
  graph = parse_input("TestCase_03_EdgeList.txt", "TestCase_03_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print(len(dfs(graph, start, end)))
  print(len(bfs(graph, start, end)))
  print(len(a_star(graph, start, end, 'euclidean')))
  print(len(a_star(graph, start, end, 'manhattan')))

Test Case 1: 25 Nodes
22
25
15
15

Test Case 2: 100 Nodes
67
76
44
38

Test Case 3: 1000 Nodes
374
980
888
857


The A* algorithms shine here in the first two test cases with low numbers of nodes, finding a path in the shortest number of steps in both. We also see breadth-first search performing the most node visits in these test cases, which is line with the expected behavior of this algorithm, with its propensity to visit a large number of nodes. The third test case is anomalous, as we see what I can only assume is depth-first search getting lucky and finding a path in one fourth the amount of steps of the other three algorithms.

**Conclusion** \
The A* algorithms would appear to be extremely well-suited for this search problem both in search time as well as number of steps, especially in the smaller search spaces. We see a drop-off in performance in the larger search space, although I am sure it finds a shorter optimal path than the other algorithms. If I were to choose a search algorithm for such a problem, I would choose A*.


\

Below is the output of each algorithm on each test case:

In [46]:
if __name__ == "__main__":

  print("\nTest Case 1:\n")
  graph = parse_input("TestCase_01_EdgeList.txt", "TestCase_01_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print(dfs(graph, start, end))
  print(bfs(graph, start, end))
  print(a_star(graph, start, end, 'euclidean'))
  print(a_star(graph, start, end, 'manhattan'))

  print("\nTest Case 2:\n")
  graph = parse_input("TestCase_02_EdgeList.txt", "TestCase_02_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print(dfs(graph, start, end))
  print(bfs(graph, start, end))
  print(a_star(graph, start, end, 'euclidean'))
  print(a_star(graph, start, end, 'manhattan'))

  print("\nTest Case 3:\n")
  graph = parse_input("TestCase_03_EdgeList.txt", "TestCase_03_NodeID.csv")
  start = graph.get_nodes()[0]
  end = graph.get_nodes()[-1]
  print(dfs(graph, start, end))
  print(bfs(graph, start, end))
  print(a_star(graph, start, end, 'euclidean'))
  print(a_star(graph, start, end, 'manhattan'))


Test Case 1:

['N_0', 'N_1', 'N_2', 'N_3', 'N_6', 'N_5', 'N_10', 'N_11', 'N_16', 'N_15', 'N_20', 'N_21', 'N_7', 'N_12', 'N_13', 'N_8', 'N_9', 'N_4', 'N_14', 'N_18', 'N_19', 'N_24']
['N_0', 'N_1', 'N_2', 'N_6', 'N_3', 'N_5', 'N_7', 'N_10', 'N_12', 'N_11', 'N_15', 'N_13', 'N_17', 'N_16', 'N_20', 'N_8', 'N_14', 'N_18', 'N_22', 'N_21', 'N_9', 'N_19', 'N_23', 'N_4', 'N_24']
['N_0', 'N_1', 'N_6', 'N_2', 'N_7', 'N_12', 'N_3', 'N_13', 'N_17', 'N_18', 'N_14', 'N_19', 'N_22', 'N_23', 'N_24']
['N_0', 'N_1', 'N_2', 'N_3', 'N_6', 'N_7', 'N_12', 'N_13', 'N_14', 'N_17', 'N_18', 'N_19', 'N_22', 'N_23', 'N_24']

Test Case 2:

['N_0', 'N_10', 'N_11', 'N_1', 'N_2', 'N_3', 'N_4', 'N_13', 'N_14', 'N_23', 'N_22', 'N_24', 'N_25', 'N_15', 'N_16', 'N_6', 'N_5', 'N_26', 'N_34', 'N_35', 'N_36', 'N_37', 'N_27', 'N_17', 'N_18', 'N_8', 'N_7', 'N_9', 'N_19', 'N_29', 'N_39', 'N_28', 'N_38', 'N_48', 'N_46', 'N_47', 'N_56', 'N_57', 'N_58', 'N_59', 'N_49', 'N_67', 'N_68', 'N_69', 'N_78', 'N_77', 'N_87', 'N_86', 'N_76',

Acknowledgements: \
Claude was utilized to discuss how to best represent the graph structure in my program. \
\
https://www.geeksforgeeks.org/a-search-algorithm/ was used for high-level implementation of the A* algorithm as well as search heuristic ideas.